# Introduction

<img src='../images/Screen Shot 2019-04-11 at 11.01.39 AM.png' />

# Random Testing In The Bigger Picture

<img src='../images/Screen Shot 2019-04-11 at 11.05.34 AM.png' />

<img src='../images/Screen Shot 2019-04-11 at 11.08.16 AM.png' />

# How Random Testing Should Work

Software systems that are not taking advantage of random testing are fundamentally flawed. If you're only testing for things you're looking for based on what you'd expect, you'll miss out on a lot of unknown unknowns.

Once we get past unit tests, we want to try to set up random system tests as quickly as possible. This will allow us to spot weak flaws easily and will (hopefully) help us feed a steady stream of bugs to our developers. It's much better to present a few bugs at a time than a heavy flow at the end. The former can be analyzed and taken care of properly while the latter will most likely be triaged and patched. By coevolving our tester and SUT together over time, we can develop a system that is very robust.

<img src='../images/Screen Shot 2019-04-11 at 11.13.29 AM.png' />

# Tuning Rules and Probabilities

Apply the process you've used for journaling to testing. The first solution rarely comes out completely formed. Reflection, iteration, 
and experimentation are key for getting to a final product.

<img src='../images/Screen Shot 2019-04-11 at 12.13.54 PM.png' />


# Filesystem Testing

<img src='../images/Screen Shot 2019-04-11 at 12.19.26 PM.png' />


# Fuzzing the Bounded Queue

<img src='../images/Screen Shot 2019-04-11 at 12.28.14 PM.png' />


# Testing Bitwise Functions

Sometimes, we already have a function that produces the correct output, but it is too slow to be feasible. We may want to create a faster implementation. The issue is that the faster code may be more complicated to write. This is a prime opportunity for randomized testing to make sure the new algorithm does what it says it does.

<img src='../images/Screen Shot 2019-04-11 at 12.29.53 PM.png' />

<img src='../images/Screen Shot 2019-04-11 at 12.36.34 PM.png' />

In this case, we used generative random testing to get inputs a and b, which led to incomplete code coverage. Switching to mutation-based random testing led to a higher probability of having a == b, which increased our code coverage from 89% to 100%. This doesn't necessarily mean that mutation-based random testing will always dominate, but that thinking critically about our testing implementation can help us create better tests.

<img src='../images/Screen Shot 2019-04-11 at 12.41.00 PM.png' />

<img src='../images/Screen Shot 2019-04-11 at 12.44.22 PM.png' />

# Fuzzing Implicit Inputs

<img src='../images/Screen Shot 2019-04-11 at 12.53.34 PM.png' />


# Can Random Testing Inspire Confidence?

<img src='../images/Screen Shot 2019-04-11 at 12.57.09 PM.png' />


# Tradeoffs in Random Testing

Pluses | Deltas
--- | ---
**Less tester bias:** By using random testing, we eliminate a lot of assumptions and it's easier to let the data speak for itself. | **Input validity:** It takes careful thought to come up with valid input for our SUT. If we don't establish proper input validity, we won't penetrate higher-order functionality.
**Low Human Cost:** Once testing is automated, the human cost of testing goes to nearly zero. | **Oracles:** We need to compare our randomized inputs to ground truth and getting there is often difficult.
**Often Surprises Us:** Random testers are inherently good at telling us things we didn't know about SUT. | **Stopping Criteria:** Often, there is no upward bound for the amount of time necessary to identify any bugs. Even if no bugs show up after running your testing suite for a day, it's still possible that a few more hours might generate your first.
**Different Fuzzers find Different Bugs:** The purpose of writing tests is to identify vulnerabilities, so by creating more than one, we can cover more ground faster. | **Unimportant Bugs:** Some bugs have such a small impact that it's not worth the effort to fix them.
**Fun!:** Automation is fun, as is exploration. Enjoy the process! | **Boring Tests:** By neglecting input validity and critical thinking, we end up waisting our time.
 | **Different Fuzzers find Different Bugs:** A simple fuzzer may find bugs that a more complicated fuzzer might miss and vice-versa. This is why iteration is so critical.

<img src='../images/Screen Shot 2019-04-11 at 1.25.10 PM.png' />


In [17]:
import math
import random
import string
import subprocess
import time

file_list = ["../data/poker_bot_notes.txt", "../data/portfolio todo.txt"]

apps = [
    "/Applications/Sublime Text.app/Contents/MacOS/Sublime Text",
    "/Applications/TextEdit.app/Contents/MacOS/TextEdit"
]

fuzz_output = "../data/fuzz.txt"

FUZZ_FACTOR = 300
NUM_TESTS = 100

for i in range(NUM_TESTS):
    file_choice = random.choice(file_list)
    app = random.choice(apps)
    
    buf = bytearray(open(file_choice, 'rb').read())
    
    num_writes = random.randrange(math.ceil((float(len(buf)) / FUZZ_FACTOR))) + 1
    
    for j in range(num_writes):
        rbyte = random.randrange(256)
        rn = random.randrange(len(buf))
        buf[rn] = "{}".format(chr(rbyte))
        
    with open(fuzz_output, 'wb') as f:
        f.write(buf)

#     open(fuzz_output, 'wb').write(buf)
        
    process = subprocess.Popen([app, fuzz_output])
    time.sleep(1)
    crashed = process.poll()
    if not crashed:
        process.terminate()

KeyboardInterrupt: 